### Постановка задачи
Построить простую модель энергопотребления здания по среднему значению, 
оценить эффективность модели через метрику
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}
* n - число наблюдений
* log - натуральный логарифм
* p_i - вычисленное значение метрики
* a_i - заданное значение метрики

Данные: http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/


### Загрузка библиотек

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams 
rcParams['figure.figsize'] = 16, 8

### Загрузка данных
Дополнительно сразу отсечем пустые дни и выделим час из значения времени

In [2]:
energy_0 = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz")
energy_0 = energy_0[energy_0["meter_reading"] > 0]
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
energy_0["hour"] = energy_0["timestamp"].dt.hour
print (energy_0.head())

      building_id  meter           timestamp  meter_reading  hour
704             0      0 2016-01-30 08:00:00        43.6839     8
725             0      0 2016-01-31 05:00:00        37.5408     5
737             0      0 2016-01-31 17:00:00        52.5571    17
2366            0      0 2016-04-08 14:00:00        59.3827    14
2923            0      0 2016-05-01 19:00:00       448.0000    19


### Разделение данных на обучение и проверку
Выделим 20% всех данных на проверку, остальные оставим на обучение

In [3]:
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2)
print (energy_0_train.head())

      building_id  meter           timestamp  meter_reading  hour
6493            0      0 2016-09-27 13:00:00        252.547    13
5101            0      0 2016-07-31 13:00:00        293.501    13
8714            0      0 2016-12-29 02:00:00        238.214     2
5746            0      0 2016-08-27 10:00:00        235.483    10
4251            0      0 2016-06-26 03:00:00        244.357     3


### Создадим модели
Среднее и медианное значение потребление энергии по часам

In [4]:
energy_0_train_hours = energy_0_train.groupby("hour")
energy_0_train_averages = pd.DataFrame(
    {"Среднее": energy_0_train_hours.mean()["meter_reading"],
     "Медиана": energy_0_train_hours.median()["meter_reading"]})
print (energy_0_train_averages)

         Среднее   Медиана
hour                      
0     240.332234  241.6260
1     238.144151  240.2610
2     238.564363  244.3570
3     241.602947  246.4040
4     239.553652  245.0390
5     239.297575  246.4040
6     239.322219  246.4040
7     239.330129  245.7220
8     242.685211  239.5790
9     238.029577  235.4830
10    234.526964  237.5310
11    233.003681  240.9440
12    236.306272  243.6740
13    236.321527  243.6740
14    240.649721  245.3805
15    235.676706  242.3090
16    234.919457  242.3090
17    234.453693  240.9440
18    237.279101  239.5790
19    236.291971  237.8725
20    239.735319  238.2140
21    235.087564  236.8480
22    239.151190  238.2140
23    238.366503  239.5790


### Функция проверки модели
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}

Для вычисления метрики создадим шесть новых столбцов в тестовом наборе данных: с логарифмом значения метрики, предсказанием по среднему и по медиане, а также с квадратом разницы предсказаний и логарифма значения. Последний столбец добавим, чтобы сравнить предсказание с его отсутствием - нулями в значениях.

In [5]:
def calculate_model (x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_0_train_averages["Среднее"][x.hour] + 1)
    meter_reading_median = np.log(energy_0_train_averages["Медиана"][x.hour] + 1)
    x["meter_reading_mean_q"] = (meter_reading_log - meter_reading_mean)**2
    x["meter_reading_median_q"] = (meter_reading_log - meter_reading_median)**2
    x["meter_reading_zero_q"] = (meter_reading_log)**2
    return x

energy_0_test = energy_0_test.apply(calculate_model,
                                    axis=1, result_type="expand")
print (energy_0_test.head())

      building_id  meter           timestamp  meter_reading  hour  \
6582            0      0 2016-10-01 06:00:00        249.135     6   
7328            0      0 2016-11-01 08:00:00        221.832     8   
3563            0      0 2016-05-28 11:00:00        174.735    11   
4352            0      0 2016-06-30 08:00:00        246.404     8   
3811            0      0 2016-06-07 19:00:00        244.357    19   

      meter_reading_mean_q  meter_reading_median_q  meter_reading_zero_q  
6582              0.001602                0.000121             30.492493  
7328              0.008003                0.005872             29.229357  
3563              0.082002                0.102227             26.718325  
4352              0.000229                0.000783             30.371370  
3811              0.001117                0.000717             30.279865  


Теперь остается просуммировать квадраты расхождений, разделить на количество значений и извлечь квадратный корень

In [7]:
energy_0_test_median_rmsle = np.sqrt(energy_0_test["meter_reading_median_q"].sum() / len(energy_0_test))
energy_0_test_mean_rmsle = np.sqrt(energy_0_test["meter_reading_mean_q"].sum() / len(energy_0_test))
energy_0_test_zero_rmsle = np.sqrt(energy_0_test["meter_reading_zero_q"].sum() / len(energy_0_test))
print ("Качество медианы:", energy_0_test_median_rmsle)
print ("Качество среднего:", energy_0_test_mean_rmsle)
print ("Качество нуля:", energy_0_test_zero_rmsle)

Качество медианы: 0.25660693484905556
Качество среднего: 0.2544643484343353
Качество нуля: 5.452515946798339
